## Deep Learning Model - CNN Multi-Label Text Classification
### Universal Sentence Encoder with Sub Themes (Right Now Ran for Main Themes)

In [79]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K


import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import spacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, hamming_loss

### Loading Data

In [2]:
X_train_Q1 = pd.read_excel('../data/interim/X_train_Q1_clean.xlsx')
X_valid_Q1 = pd.read_excel('../data/interim/X_valid_Q1_clean.xlsx')

y_train_Q1 = pd.read_excel('../data/interim/y_train_Q1.xlsx')
y_valid_Q1 = pd.read_excel('../data/interim/y_valid_Q1.xlsx')

### Creating a Unified Dataframe for CNN Ready Model

In [3]:
df = pd.concat([X_train_Q1, y_train_Q1.iloc[:,0:12]], axis = 1)

In [4]:
df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0


In [5]:
data_df = df

In [6]:
data_df.shape

(10376, 13)

### Pre-processing

#### General

In [6]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [7]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [8]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in data_df['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
data_df['preprocessed_comments']=preprocessed_synopsis

In [9]:
data_df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH,preprocessed_comments
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0,real diversity you need create seats table mea...
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0,keep building warmer provide warm water bathroom
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0,better communication top
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0,would beneficial management not micro manage
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0,education applicable job


### Splitting into Train and Test

In [10]:
X_train = data_df[['preprocessed_comments']]
y_train = data_df.drop(['Comment', 'preprocessed_comments'], axis=1)

In [11]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

### Preparing Labels

**Max length of sentence**

In [ ]:
def max_len(x):
    a=x.split()
    return len(a)

max_len = max(data_df['Comment'].apply(max_len))
max_len

### Vocab Size

In [ ]:
vect=Tokenizer()
vect.fit_on_texts(X_train['preprocessed_comments'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

**Universal Sentence Encoder**

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
embeddings = embed(X_train['preprocessed_comments'])

In [ ]:
embedding_matrix = np.array(embeddings)

In [ ]:
embedding_matrix

In [ ]:
embedding_matrix.shape

## Modelling CNN

#### Defining Model

## Universal Sentence Encoder

In [ ]:
y_train = np.array(y_train)

max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 1
embed_size = 512 # for universal sentence encoder
n_class = 12

In [ ]:
model = Sequential()

model.add(Embedding(max_features, embed_size, weights=[embedding_matrix],
                        trainable=False, input_length=maxlen))




model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu',
                 strides=1))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(n_class, activation = 'sigmoid'))


model.summary()

In [ ]:
padded_docs_train.shape

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

# Train Model
model.fit(padded_docs_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.05)

### Testing Simple Keras Model

In [ ]:
max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 150
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 20
embed_size = 512 # for universal sentence encoder
n_class = 12

In [ ]:
model = Sequential()

model.add(Dense(max_features, input_shape=(512,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(max_features, input_shape=(512,)))
#model.add(Dropout(0.1))
model.add(Dense(n_class))
model.add(Activation('sigmoid'))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(embedding_matrix, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.15)

## Evaluation - Validation Set

In [ ]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

# creating X and Y
X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

y_valid = np.array(y_valid)

# creating embedding for validation
embeddings_valid = embed(X_valid['preprocessed_comments'])
embedding_matrix_valid = np.array(embeddings_valid)

score = model.evaluate(embedding_matrix_valid,y_valid)
score

## Evaluation - Precision & Recall

In [ ]:
pred = model.predict(embedding_matrix_valid, batch_size=batch_size, verbose=1)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

#predictions=model.predict([padded_docs_test])
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print('\n')

### Testing RNN

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[512])
    layer = Embedding(max_features,50,input_length=512)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(12,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(embedding_matrix, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.05)

## New Model Testing ends

**Score on Validation Data**

In [ ]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

# creating X and Y

X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

y_valid = np.array(y_valid)

# creating padded dataset for x_valid
encoded_docs_valid = vect.texts_to_sequences(X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')
print(padded_docs_valid)

In [ ]:
score = model.evaluate(padded_docs_valid,y_valid)

In [ ]:
score

## Saving Model

In [ ]:
model.save('../models/cnn_use_sub_themes.pkl')

## Precision & Recall

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

pred = model.predict(padded_docs_valid, batch_size=batch_size, verbose=1)

In [ ]:
#predictions=model.predict([padded_docs_test])
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

## New Way of Running USE

In [42]:
X_train_array = np.asarray(X_train['preprocessed_comments'])
y_train_array = np.asarray(y_train)

In [43]:
X_valid_array = np.asarray(X_valid['preprocessed_comments'])
y_valid_array = np.asarray(y_valid)

In [44]:
#max_features = embedding_matrix.shape[0]
#maxlen = max_len
batch_size = 150
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 1
embed_size = 512 # for universal sentence encoder
n_class = 12

In [45]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
hub_layer = hub.KerasLayer(model, output_shape=[512], input_shape=[], dtype=tf.string, trainable=True)

In [30]:
hub_layer(X_train_array[:4])

<tf.Tensor: shape=(4, 512), dtype=float32, numpy=
array([[-0.0366986 , -0.07417478,  0.03231421, ...,  0.01621996,
         0.00728293,  0.00265465],
       [ 0.05218185, -0.01258107, -0.04657765, ...,  0.05539098,
         0.05895474, -0.01712018],
       [-0.01777639, -0.04568463,  0.00473103, ..., -0.00073466,
        -0.08261821,  0.05859691],
       [ 0.04762139, -0.06028655,  0.06576026, ...,  0.04358777,
        -0.08644933, -0.02078426]], dtype=float32)>

In [46]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(200, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(100, activation = 'sigmoid'))
model.add(tf.keras.layers.Dense(12, activation = 'sigmoid'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 512)               256797824 
_________________________________________________________________
dense_32 (Dense)             (None, 60)                30780     
_________________________________________________________________
dense_33 (Dense)             (None, 500)               30500     
_________________________________________________________________
dropout_16 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 200)               100200    
_________________________________________________________________
dropout_17 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 100)              

In [47]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [48]:
history = model.fit(X_train_array,
                    y_train_array,
                    epochs=50,
                    batch_size=512,
                    validation_data=(X_valid_array, y_valid_array),
                    verbose=1)

Train on 10376 samples, validate on 2594 samples
Epoch 1/50
10376/10376 [==============================] - 42s 4ms/sample - loss: 0.5325 - accuracy: 0.7184 - val_loss: 0.3565 - val_accuracy: 0.8838
Epoch 2/50
10376/10376 [==============================] - 37s 4ms/sample - loss: 0.3536 - accuracy: 0.8830 - val_loss: 0.3498 - val_accuracy: 0.8838
Epoch 3/50
10376/10376 [==============================] - 39s 4ms/sample - loss: 0.3523 - accuracy: 0.8830 - val_loss: 0.3492 - val_accuracy: 0.8838
Epoch 4/50
10376/10376 [==============================] - 37s 4ms/sample - loss: 0.3520 - accuracy: 0.8830 - val_loss: 0.3490 - val_accuracy: 0.8838
Epoch 5/50
10376/10376 [==============================] - 37s 4ms/sample - loss: 0.3520 - accuracy: 0.8830 - val_loss: 0.3491 - val_accuracy: 0.8838
Epoch 6/50
10376/10376 [==============================] - 36s 3ms/sample - loss: 0.3518 - accuracy: 0.8830 - val_loss: 0.3489 - val_accuracy: 0.8838
Epoch 7/50
10376/10376 [==============================] -

In [49]:
pred = model.predict(X_valid_array, batch_size=512, verbose=1)
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid_array, pred, average='micro')
    recall = recall_score(y_valid_array, pred, average='micro')
    f1 = f1_score(y_valid_array, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

2594/2594 [==============================] - 1s 290us/sample
For threshold:  0.1
Micro-average quality numbers
Precision: 0.2905, Recall: 0.6976, F1-measure: 0.4102
For threshold:  0.2
Micro-average quality numbers
Precision: 0.4328, Recall: 0.5558, F1-measure: 0.4867
For threshold:  0.3
Micro-average quality numbers
Precision: 0.5563, Recall: 0.4613, F1-measure: 0.5044
For threshold:  0.4
Micro-average quality numbers
Precision: 0.6053, Recall: 0.4281, F1-measure: 0.5015
For threshold:  0.5
Micro-average quality numbers
Precision: 0.7196, Recall: 0.3469, F1-measure: 0.4681
For threshold:  0.6
Micro-average quality numbers
Precision: 0.7335, Recall: 0.3195, F1-measure: 0.4451
For threshold:  0.7
Micro-average quality numbers
Precision: 0.7729, Recall: 0.2869, F1-measure: 0.4185
For threshold:  0.8
Micro-average quality numbers
Precision: 0.9182, Recall: 0.1985, F1-measure: 0.3264
For threshold:  0.9
Micro-average quality numbers
Precision: 0.9529, Recall: 0.1567, F1-measure: 0.2692


In [33]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [75]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [76]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(200, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(100, activation = 'sigmoid'))
model.add(tf.keras.layers.Dense(12, activation = 'sigmoid'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_6 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_54 (Dense)             (None, 60)                1260      
_________________________________________________________________
dense_55 (Dense)             (None, 500)               30500     
_________________________________________________________________
dropout_26 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_56 (Dense)             (None, 200)               100200    
_________________________________________________________________
dropout_27 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 100)              

In [77]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [78]:
history = model.fit(X_train_array,
                    y_train_array,
                    epochs=300,
                    batch_size=258,
                    validation_data=(X_valid_array, y_valid_array),
                    verbose=1)

Train on 10376 samples, validate on 2594 samples
Epoch 1/300
10376/10376 [==============================] - 1s 119us/sample - loss: 0.4100 - accuracy: 0.8458 - val_loss: 0.3442 - val_accuracy: 0.8838
Epoch 2/300
10376/10376 [==============================] - 0s 41us/sample - loss: 0.3357 - accuracy: 0.8849 - val_loss: 0.3213 - val_accuracy: 0.8925
Epoch 3/300
10376/10376 [==============================] - 0s 39us/sample - loss: 0.3173 - accuracy: 0.8923 - val_loss: 0.3064 - val_accuracy: 0.8934
Epoch 4/300
10376/10376 [==============================] - 0s 39us/sample - loss: 0.3027 - accuracy: 0.8934 - val_loss: 0.2940 - val_accuracy: 0.8937
Epoch 5/300
10376/10376 [==============================] - 1s 53us/sample - loss: 0.2887 - accuracy: 0.8941 - val_loss: 0.2810 - val_accuracy: 0.8947
Epoch 6/300
10376/10376 [==============================] - 1s 58us/sample - loss: 0.2738 - accuracy: 0.8977 - val_loss: 0.2679 - val_accuracy: 0.9012
Epoch 7/300
10376/10376 [=========================

In [82]:
pred = model.predict(X_valid_array, batch_size=batch_size, verbose=1)

predictions = pred

predictions_results = []
thresholds=[0.5,0.6,0.7,0.8,0.9] #np.arange(.5, 1, 0.1).tolist()
for val in thresholds:
    pred=predictions.copy()
    pred[pred>=val]=1
    pred[pred<val]=0
    accuracy = accuracy_score(y_valid_array, pred, normalize=True, sample_weight=None)#average='micro')
    hamming = hamming_loss(y_valid_array, pred)
    precision = precision_score(y_valid_array, pred, average='micro')
    recall = recall_score(y_valid_array, pred, average='micro')
    f1 = f1_score(y_valid_array, pred, average='micro')
    case= {'Threshold': val,
           'Accuracy': accuracy,
           'Hamming loss': hamming,
           'Precision': precision,
           'Recall': recall,
           'F1-measure': f1}
    predictions_results.append(case)
print("Results on y prediction:")
pd.DataFrame(predictions_results)

2594/2594 [==============================] - 0s 16us/sample
Results on y prediction:


,Threshold,Accuracy,Hamming loss,Precision,Recall,F1-measure
0,0.5,0.391288,0.097758,0.578962,0.582642,0.580796
1,0.6,0.395914,0.096826,0.584641,0.576562,0.580573
2,0.7,0.398227,0.096055,0.590386,0.566888,0.578398
3,0.8,0.399769,0.094834,0.598057,0.561360,0.579127
4,0.9,0.405551,0.093132,0.610310,0.549751,0.578450


## New Approach

In [64]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
hub_layer = hub.KerasLayer(model, output_shape=[512], input_shape=[], dtype=tf.string, trainable=True)

In [70]:
input = tf.keras.layers.Input(shape=(), name="Input", dtype=tf.string)
x = hub_layer(input)
x = tf.keras.layers.Reshape(input_shape=(512,), target_shape=(512, 1))(x)
x = tf.keras.layers.Conv1D(128, 2, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling1D(5, padding='same')(x)
x = tf.keras.layers.Conv1D(128, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling1D(5, padding='same')(x)
x = tf.keras.layers.Conv1D(128, 4, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling1D(40, padding='same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(12, activation='sigmoid')(x)
m = tf.keras.Model(input, output)
m.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None,)]                 0         
_________________________________________________________________
keras_layer_5 (KerasLayer)   (None, 512)               256797824 
_________________________________________________________________
reshape_5 (Reshape)          (None, 512, 1)            0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 512, 128)          384       
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 103, 128)          0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 103, 128)          49280     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 21, 128)           0   

In [72]:
m.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [73]:
history = m.fit(X_train_array,
                    y_train_array,
                    epochs=5,
                    batch_size=258,
                    validation_data=(X_valid_array, y_valid_array),
                    verbose=1)

Train on 10376 samples, validate on 2594 samples
Epoch 1/5
10376/10376 [==============================] - 81s 8ms/sample - loss: 0.4552 - accuracy: 0.8634 - val_loss: 0.3497 - val_accuracy: 0.8838
Epoch 2/5
10376/10376 [==============================] - 80s 8ms/sample - loss: 0.3567 - accuracy: 0.8830 - val_loss: 0.3466 - val_accuracy: 0.8838
Epoch 3/5
10376/10376 [==============================] - 75s 7ms/sample - loss: 0.3522 - accuracy: 0.8830 - val_loss: 0.3447 - val_accuracy: 0.8838
Epoch 4/5
10376/10376 [==============================] - 76s 7ms/sample - loss: 0.3488 - accuracy: 0.8830 - val_loss: 0.3424 - val_accuracy: 0.8838
Epoch 5/5
10376/10376 [==============================] - 77s 7ms/sample - loss: 0.3434 - accuracy: 0.8830 - val_loss: 0.3388 - val_accuracy: 0.8838


In [74]:
pred = m.predict(X_valid_array, batch_size=512, verbose=1)
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid_array, pred, average='micro')
    recall = recall_score(y_valid_array, pred, average='micro')
    f1 = f1_score(y_valid_array, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

2594/2594 [==============================] - 2s 599us/sample
For threshold:  0.1
Micro-average quality numbers
Precision: 0.1662, Recall: 0.8295, F1-measure: 0.2769
For threshold:  0.2
Micro-average quality numbers
Precision: 0.2123, Recall: 0.3007, F1-measure: 0.2489
For threshold:  0.3
Micro-average quality numbers
Precision: 0.2339, Recall: 0.1669, F1-measure: 0.1948
For threshold:  0.4
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.5
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.6
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.7
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


/Users/karan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
